In [1]:
import torch, numpy as np, networkx as nx
from torch_geometric.utils import from_networkx
from torch_geometric.nn.models import Node2Vec

In [2]:
import torch_scatter, torch_sparse, torch_cluster, pyg_lib
print("all good!")

all good!


In [3]:
import torch, platform, subprocess, sys
print("torch :", torch.__version__)
print("cuda  :", torch.version.cuda)

torch : 2.1.2+cu121
cuda  : 12.1


In [4]:
import pandas as pd 
def to_undirected(self, df):
    """
    make a directed graph into an undirected graph

    df: a dataframe where each row represents one edge from "gene1" to "gene2"
    """
    df_dup = df.reindex(columns=['gene2','gene1'])
    df_dup.columns = ['gene1','gene2']
    df = df._append(df_dup)
    df.drop_duplicates(inplace=True)

    return df
data_dir = "../data/"
network_type = "PPI"

network = pd.read_csv(data_dir + "networks/" + network_type + ".csv", index_col = 0)
            
network = network[network['Experimental System Type'] == 'physical']
network.rename(columns={'Official Symbol Interactor A':'gene1', 'Official Symbol Interactor B':'gene2'}, inplace=True)
network = network[['gene1','gene2']]

In [5]:
network

,gene1,gene2
0,MAP2K4,FLNC
1,MYPN,ACTN2
2,ACVR1,FNTA
3,GATA2,PML
4,RPA2,STAT3
...,...,...
2101421,PDLIM5,TRIM25
2101422,STUB1,PDLIM5
2101423,STUB1,SMAD3
2101424,SMAD3,STUB1


In [6]:
import torch.optim
G = nx.from_pandas_edgelist(network, source='gene1', target='gene2',
                            edge_attr=None, create_using=nx.Graph())
data = from_networkx(G)

model = Node2Vec(edge_index=data.edge_index, embedding_dim=128,
                 walk_length=20, context_size=10, p=1, q=1,
                 sparse=True)            # sparse=True → SparseAdam
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optim  = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

def train(epoch):
    model.train(); tot=0
    for pos_rw, neg_rw in loader:
        optim.zero_grad()
        loss = model.loss(pos_rw, neg_rw)
        loss.backward(); optim.step()
        tot += loss.item()
    print(f'Epoch {epoch}: {tot/len(loader):.4f}')
for e in range(1,6): train(e)

Epoch 1: 9.0836
Epoch 2: 6.8891
Epoch 3: 5.0452
Epoch 4: 3.9508


KeyboardInterrupt: 

In [6]:
python -c "import torch, sys; print(torch.__version__, torch.version.cuda)"
# → 2.1.2+cu121  (举例)


SyntaxError: invalid syntax (2765272688.py, line 1)